In [161]:
import torch as t
import torch.nn as nn
import torch.nn.functional as F

import torchvision as tv

import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import os

In [201]:
t.manual_seed(101)

img_dir = '../../datasets/leaf-classification/images/'
train_csv_dir = '../../datasets/leaf-classification/train.csv'
eval_csv_dir = '../../datasets/leaf-classification/test.csv'

img_filenames_list = os.listdir(img_dir)
eval_df = pd.read_csv(eval_csv_dir)
train_df = pd.read_csv(train_csv_dir)

In [202]:
eval_ids = eval_df['id']
eval_ids = eval_ids.to_list()
len(eval_ids)

594

In [203]:
transform = tv.transforms.Compose([
    tv.transforms.Resize((64, 64)),
    tv.transforms.Lambda(lambda x: x / 255.0)
])

def readImage(id):
    eval_img_dir = img_dir + str(id) + '.jpg'
    img_tensor = tv.io.read_image(eval_img_dir)
    img_tensor = transform(img_tensor)
    img_tensor = img_tensor.unsqueeze(0)
    return img_tensor

eval_inputs = t.cat([readImage(id) for id in eval_ids], dim=0)

In [209]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        # original image size: 64 x 64
        self.conv1 = nn.Conv2d(1, 32, 3) # -> 62 x 62
        # max pool -> 31 x 31
        self.conv2 = nn.Conv2d(32, 64, 3) # -> 29 x 29
        # max pool again -> 14 x 14
        self.conv3 = nn.Conv2d(64, 64, 3) # -> 12 x 12
        # max pool again -> 6 x 6
        self.layer1 = nn.Linear(64* 6 * 6, 128)
        self.layer2 = nn.Linear(128, 99)

        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.2)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = t.flatten(x, 1)
        x = self.dropout(x)
        x = self.layer1(x)
        x = self.relu(x)
        x = self.layer2(x)
        x = self.softmax(x)
        return x

In [210]:
model = Model()
model.load_state_dict(t.load('./models/model_final.pth'))

<All keys matched successfully>

In [211]:
pred = model.forward(eval_inputs)
pred = pred.detach().numpy()
pred.shape

(594, 99)

In [212]:
categories = train_df['species'].astype('category').cat.categories
pd.set_option('display.float_format', lambda x: '%.3f' % x)
submission_df = pd.DataFrame(columns=categories)
submission_df['id'] = eval_df['id']
for i in range(99):
    submission_df[categories[i]] = pred[:, 98-i]

In [213]:
# save the dataframe
submission_df.to_csv('submission.csv', index=False)